# MapsTo in pySBOL2
The MapsTo class in the Synthetic Biology Open Language (SBOL) is used to explicitly state that two `ComponentInstance` objects, often from different levels of design hierarchy, represent the same biological entity. It is most often used when `ModuleDefinition` and `ComponentDefinition` objects are composed using `Module` and `ComponentInstance` objects.

`MapsTo` objects define how a `ComponentInstance` in a higher-level design relates to a `ComponentInstance` in a lower-level design through identity and refinement relationships.

MapsTo objects have the following required properties:

* `local`: Refers to the `ComponentInstance` in the higher-level design.
* `remote`: Refers to the `ComponentInstance` in the lower-level design. The referenced instance must have access="public".
* `refinement`: Specifies how to interpret the relationship between the local and remote instances using a URI. For * `example`: http://sbols.org/v2#useRemote.

This example demonstrates linking a FunctionalComponent in a high-level toggle switch module to one in a lower-level LacI inverter using a MapsTo object. We will:

1. Define all necessary biological parts (`ComponentDefinition`): two proteins (LacI, cI) and two promoters (pLac, pR).
2. Define two `ModuleDefinition` objects for genetic inverters: one repressed by LacI and one repressed by cI.
3. Define a higher-level `ModuleDefinition` for the toggle switch itself.
4. Instantiate both inverters inside the toggle switch module.
5. Use `MapsTo` to connect the inputs and outputs of the inverters, forming the complete, mutually-repressive feedback loop.

For more information on the MapsTo class and its properties, refer to page 30 in the SBOL 2.x.x specification document.

In [1]:
import sbol2

In [2]:
doc = sbol2.Document()

# Set the namespace for all SBOL objects
sbol2.setHomespace('https://github.com/SynBioDex/SBOL-Notebooks')

## Define Shared Biological Parts

Here we define four `ComponentDefinition` objects representing the core parts of our toggle switch: the coding sequences for the two repressor proteins (LacI and cI) and the two promoters they regulate (pLac and pR). In a real design, these would be detailed with sequence information.

In [3]:
# Coding Sequences (CDS)
lacI_cds = sbol2.ComponentDefinition('LacI_CDS', sbol2.BIOPAX_DNA)
lacI_cds.name = 'LacI Coding Sequence'
lacI_cds.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(lacI_cds)

cI_cds = sbol2.ComponentDefinition('cI_CDS', sbol2.BIOPAX_DNA)
cI_cds.name = 'Lambda cI Coding Sequence'
cI_cds.addRole(sbol2.SO_CDS)
doc.addComponentDefinition(cI_cds)

# Define Promoters
pLac = sbol2.ComponentDefinition('pLac', sbol2.BIOPAX_DNA)
pLac.name = 'pLac Promoter'
pLac.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pLac)

pR = sbol2.ComponentDefinition('pR', sbol2.BIOPAX_DNA)
pR.name = 'pR Promoter'
pR.addRole(sbol2.SO_PROMOTER)
doc.addComponentDefinition(pR)

## Define the Inverter Modules (Lower-Level Subsystems)

A toggle switch is composed of two inverters. We define each as a `ModuleDefinition`. Each inverter has an input (the repressor protein), a regulated promoter, and an output (the protein it produces). We also add `Interaction` objects to specify the biological function: repression.

- **LacI Inverter**: Takes LacI as input to repress the pLac promoter, which produces the cI protein.
- **cI Inverter**: Takes cI as input to repress the pR promoter, which produces the LacI protein.

In [4]:
# 1. LacI Inverter Module (LacI represses production of cI)
laci_inverter_md = sbol2.ModuleDefinition('LacI_Inverter')

# Functional Components of the LacI inverter
tf_in = laci_inverter_md.functionalComponents.create('input_transcription_factor')
tf_in.definition = lacI_cds
tf_in.access = sbol2.SBOL_ACCESS_PUBLIC
tf_in.direction = sbol2.SBOL_DIRECTION_IN

prom = laci_inverter_md.functionalComponents.create('promoter')
prom.definition = pLac.identity
prom.access = sbol2.SBOL_ACCESS_PRIVATE

gene_out = laci_inverter_md.functionalComponents.create('output_gene')
gene_out.definition = cI_cds.identity
gene_out.access = sbol2.SBOL_ACCESS_PUBLIC
gene_out.direction = sbol2.SBOL_DIRECTION_OUT


# 2. cI Inverter Module (cI represses production of LacI)
ci_inverter_md = sbol2.ModuleDefinition('cI_Inverter')

# Functional Components of the cI inverter
tf_in_2 = ci_inverter_md.functionalComponents.create('input_transcription_factor')
tf_in_2.definition = cI_cds.identity
tf_in_2.access = sbol2.SBOL_ACCESS_PUBLIC
tf_in_2.direction = sbol2.SBOL_DIRECTION_IN

prom_2 = ci_inverter_md.functionalComponents.create('promoter')
prom_2.definition = pR.identity
prom_2.access = sbol2.SBOL_ACCESS_PRIVATE

gene_out_2 = ci_inverter_md.functionalComponents.create('output_gene')
gene_out_2.definition = lacI_cds.identity
gene_out_2.access = sbol2.SBOL_ACCESS_PUBLIC
gene_out_2.direction = sbol2.SBOL_DIRECTION_OUT

## Define the Toggle Switch Module (Higher-Level Circuit)

This `ModuleDefinition` represents the complete genetic toggle switch. From this high-level perspective, we only care about the two key players: the LacI protein and the cI protein.

In [5]:
toggle_md = sbol2.ModuleDefinition('toggle_switch')

# The toggle switch's behavior is defined by two protein pools
laci_fc = toggle_md.functionalComponents.create('LacI_protein')
laci_fc.definition = lacI_cds.identity
laci_fc.direction  = sbol2.SBOL_DIRECTION_IN_OUT

ci_fc = toggle_md.functionalComponents.create('cI_protein')
ci_fc.definition = cI_cds.identity
ci_fc.direction  = sbol2.SBOL_DIRECTION_IN_OUT

## Add Inverter Modules to Toggle Switch (Composition)

We now instantiate both inverter modules within the toggle switch module. This reflects **module composition** in SBOL—building a larger system from smaller, defined subsystems.

In [6]:
# Instantiate the LacI inverter
laci_inverter_instance = toggle_md.modules.create('laci_inverter_inst')
laci_inverter_instance.definition = laci_inverter_md.identity

# Instantiate the cI inverter
ci_inverter_instance = toggle_md.modules.create('ci_inverter_inst')
ci_inverter_instance.definition = ci_inverter_md.identity

## Wire the Circuit with `MapsTo`

This is the key step. We use `MapsTo` objects to connect the components and form the feedback loop. We need four mappings to declare that:
1. The `LacI_protein` in the toggle switch is the same molecule that acts as the input to the LacI inverter.
2. The `cI_protein` in the toggle switch is the same molecule that is produced as the output of the LacI inverter.
3. The `cI_protein` in the toggle switch is the same molecule that acts as the input to the cI inverter.
4. The `LacI_protein` in the toggle switch is the same molecule that is produced as the output of the cI inverter.

This wiring correctly models the mutual repression: LacI represses cI production, and cI represses LacI production.

In [7]:
# Map 1: LacI protein is the input to the LacI inverter
map1 = laci_inverter_instance.mapsTos.create('map_laci_in')
map1.refinement = sbol2.SBOL_REFINEMENT_USE_REMOTE
map1.local = laci_fc.identity
map1.remote = tf_in.identity

# Map 2: cI protein is the output of the LacI inverter
map2 = laci_inverter_instance.mapsTos.create('map_ci_out')
map2.local = ci_fc.identity
map2.remote = gene_out.identity

# Map 3: cI protein is the input to the cI inverter
map3 = ci_inverter_instance.mapsTos.create('map_ci_in')
map3.local = ci_fc.identity
map3.remote = tf_in_2.identity

# Map 4: LacI protein is the output of the cI inverter
map4 = ci_inverter_instance.mapsTos.create('map_laci_out')
map4.local = laci_fc.identity
map4.remote = gene_out_2.identity

## Finalize and Validate

We add all high-level modules to the document and check that the complete design conforms to SBOL specifications. If valid, it is saved as `toggle_switch_complete.xml`.

In [8]:
doc.addModuleDefinition(laci_inverter_md)
doc.addModuleDefinition(ci_inverter_md)
doc.addModuleDefinition(toggle_md)

# Validate the document
report = doc.validate()

if report == "Valid.":
    doc.write("example_mapsto.xml")
else:
    print(report)